# National NO<sub>2</sub> Convolutional Neural Network Model 

In this notebook, we provide an example of a simple train test split using our PyTorch based model architecture. The NO<sub>2</sub> measurements are first obtained from the 'daily measurements.csv' file.

In [ ]:
# import necessary modules

import torch
import pandas as pd
from torch.utils.data import Subset
from torch.utils.data.dataloader import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
import torch.nn.functional as F
from torch import nn
from torchmetrics.functional import r2_score, mean_absolute_error
from tqdm.notebook import tqdm

In [ ]:
# specify cuda capable GPU

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
df_all = pd.read_csv('data/annual measurements.csv')

In [ ]:
site_nums = df_all['Monitoring Num'].unique().tolist()

In [ ]:
class TorchStandardScaler:
    def fit(self, x, dim):
        self.mean = x.mean(dim, keepdim=True)
        self.std = x.std(dim, unbiased=False, keepdim=True)
    def transform(self, x):
        x -= self.mean
        x /= (self.std + 1e-5)
        return x

In [ ]:
encoder = OneHotEncoder()
encoder.fit(df_all['State'].unique().reshape(-1, 1))

## Load data into memory

Next, we load data files into memory. Each file is formatted as a dictionary saved as a NumPy file. Each file loads data at different temporal frequencies. We convert each array to a Tensor and unsqueeze it. After, that we take data from these dictionaries to form the actual samples list, called 'items.'

In [ ]:
era5 = np.load('data/annual era5.npy', allow_pickle=True).item()
omi_no2 = np.load('data/annual no2.npy', allow_pickle=True).item()
roas = np.load('data/road images.npy', allow_pickle=True).item()
eles = np.load('data/elevation.npy', allow_pickle=True).item()
firs = np.load('data/annual fires.npy', allow_pickle=True).item()
ndvs = np.load('data/annual ndvi.npy', allow_pickle=True).item()
pops = np.load('data/pop density.npy', allow_pickle=True).item()
plas = np.load('data/power plants.npy', allow_pickle=True).item()
tres = np.load('data/tree cover.npy', allow_pickle=True).item()
wats = np.load('data/water.npy', allow_pickle=True).item()
wels = np.load('data/wells.npy', allow_pickle=True).item()
rais = np.load('data/railway images.npy', allow_pickle=True).item()
imps = np.load('data/impervious.npy', allow_pickle=True).item()

# one image for all data
for num in tqdm(site_nums):
    roas[num] = torch.Tensor(np.concatenate(list(roas[num].values())))
    eles[num] = torch.Tensor(eles[num]).unsqueeze(dim=0)
    wats[num] = torch.Tensor(wats[num]).unsqueeze(dim=0)
    pop = torch.Tensor(pops[num]).unsqueeze(dim=0)
    pop[pop < 0] = 0
    pops[num] = pop
    pla = torch.Tensor(np.stack(list(plas[num].values())))
    pla[pla > 0] = 1
    plas[num] = pla
    wels[num] = torch.Tensor(wels[num]).unsqueeze(dim=0)
    imps[num] = torch.Tensor(imps[num]).unsqueeze(dim=0)
    rais[num] = torch.Tensor(rais[num]).unsqueeze(dim=0)

# yearly images
for num in tqdm(site_nums):
    for year in list(tres[num]):
        tres[num][year] = torch.Tensor(tres[num][year])
        tres[num][2021] = tres[num][2020]
        tres[num][2022] = tres[num][2020]

# monthly images
for num in tqdm(site_nums):
    year_info = firs[num]
    for year in list(year_info):
        firs[num][year] = torch.Tensor(year_info[year]).unsqueeze(dim=0)
    year_info = ndvs[num]
    for year in list(year_info):
        ndvs[num][year] = torch.Tensor(year_info[year]).unsqueeze(dim=0)

In [ ]:
dataset = []

for index in tqdm(range(len(df_all))):
    coors = np.array([df_all.loc[index, 'Latitude'], df_all.loc[index, 'Longitude']])
    state = encoder.transform([[df_all.loc[index, 'State']]]).toarray()[0]
    num = df_all.loc[index, 'Monitoring Num']
    year = df_all.loc[index, 'Year']
    time_data = np.array([year])

    road = roas[num]
    elevation = eles[num]
    fires = firs[num][year]
    ndvi = ndvs[num][year]
    pop = pops[num]
    plants = plas[num]
    tree = tres[num][year]
    water = wats[num]
    wells = wels[num]
    rails = rais[num]
    impervious = imps[num]
    no2 = torch.Tensor(omi_no2[num][year]).unsqueeze(dim=0)

    era = era5[num][year]
    nums = torch.Tensor(np.concatenate([era, state, coors, time_data])).to(torch.float)

    label = torch.Tensor([df_all.loc[index, 'Annual Measurement']]).to(torch.float)

    tens = torch.cat([road, elevation, fires, ndvi, no2, pop, plants, tree, water, wells, rails, impervious]).to(torch.float)

    dataset.append((tens, nums, label))

In [ ]:
def fit_scaler(dataset, train_indices, dim):
    train_img = []
    train_num = []
    for x in tqdm(train_indices):
        samp = dataset[x]
        train_img.append(samp[0])
        train_num.append(samp[1])

    train_img = torch.stack(train_img)
    train_num = torch.stack(train_num)

    img_scaler = TorchStandardScaler()
    img_scaler.fit(train_img, dim)

    num_scaler = TorchStandardScaler()
    num_scaler.fit(train_num, 0)
    
    return img_scaler, num_scaler

In [ ]:
train_indices, test_indices = train_test_split(list(range(len(dataset))))

In [ ]:
img_scaler, num_scaler = fit_scaler(dataset, train_indices, dim=(0,2,3))

# do not scale for one hot encoded data

for num in [10, 11, 12, 13, 18]:
    img_scaler.mean[:, num] = 0.00001
    img_scaler.std[:, num] = 1.00001

for num in range(17, 64):
    num_scaler.mean[:, num] = 0.00001
    num_scaler.std[:, num] = 1.00001

In [ ]:
batch_size = 64

train_data = Subset(dataset, train_indices)
test_data = Subset(dataset, test_indices)
train_dl = DataLoader(train_data, batch_size, shuffle=True)
test_dl = DataLoader(test_data, batch_size, shuffle=True)

## Model Architectures

To build our model architecture, we first build a base class called RegressionBase in which the primary CNN class will extend. The primary CNN contains two parts: a CNN and a feedforward network. First, the CNN processes the image data. Then, the numerical data is concatenated with the CNN selected features, before being processed into the feedforward network.

In [ ]:
# r2: r-squared score
# mae: mean absolute error

class RegressionBase(nn.Module):
    
    def training_step(self, batch):
        images, nums, labels = batch 
        labels = labels.to(device)
        out = self(img_scaler.transform(images.to(device)), num_scaler.transform(nums.to(device)))
        loss = F.mse_loss(out, labels)
        acc = r2_score(out, labels)
        return loss, acc
    
    def validation_step(self, batch):
        images, nums, labels = batch
        labels = labels.to(device)
        out = self(img_scaler.transform(images.to(device)), num_scaler.transform(nums.to(device)))
        loss = F.mse_loss(out, labels)
        acc = r2_score(out, labels)
        mae = mean_absolute_error(out, labels)
        return loss.detach(), acc, mae

In [ ]:
class CNN(RegressionBase):
    def __init__(self):
        super().__init__()
        self.CNN = nn.Sequential(
            
            nn.Conv2d(21, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten()
        )
        
        self.fc1 = nn.Linear(3907, 1500)
        self.dp1  = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(1500, 500)
        self.dp2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(500,1)
    
    def forward(self, image, data):
        x1 = self.CNN(image)
        x2 = data
        
        x = torch.cat((x1, x2), dim=1)
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = F.relu(self.fc2(x))
        x = self.dp2(x)
        x = self.fc3(x)
        return x
        
model = CNN()
model.to(device)

In [ ]:
def evaluate(model, val_dl):
    with torch.no_grad():
        model.eval()
        res = [model.validation_step(b) for b in val_dl]
        losses = [x[0] for x in res]
        epoch_loss = torch.stack(losses).mean()
        accs = [x[1] for x in res]
        epoch_acc = torch.stack(accs).mean()
        mae = [x[2] for x in res]
        epoch_mae = torch.stack(mae).mean()
    return {'loss': epoch_loss.item(), 'acc': epoch_acc.item(), 'mae': epoch_mae.item()}

In [ ]:
# send scaler to GPU

img_scaler.mean = img_scaler.mean.to(device)
img_scaler.std = img_scaler.std.to(device)
num_scaler.mean = num_scaler.mean.to(device)
num_scaler.std = num_scaler.std.to(device)

In [ ]:
# train model

lr = 0.0005
epochs = 300

performances = []
optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9)

for epoch in range(1, epochs + 1):
    progress = tqdm(train_dl)
    model.train()
    train_losses = []
    train_accs = []
    for _, batch in enumerate(progress):
        loss, acc = model.training_step(batch)
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        train_losses.append(loss.item())
        train_accs.append(acc.item())
        
        progress.set_description(f'Epoch [{epoch}/{epochs}]')
        progress.set_postfix(loss=np.mean(train_losses), acc=np.mean(train_accs))
        

    result = evaluate(model, test_dl)
    result['train loss'] = np.mean(train_losses)
    performances.append(result)
    
    print("Epoch [{}/{}], train loss: {:.4f}, val loss: {:.4f}, acc: {:.4f}, mae: {:.4f}".format(epoch, epochs, result['train loss'], result['loss'], result['acc'], result['mae']))